In [11]:
import numpy as np
import pandas as pd
import os
import requests 
from multiprocessing.pool import ThreadPool
import time
import random
from shutil import copyfile
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [2]:
#path for FIFI cleaned and preprocessed data frame
path_FIFI = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/generated_datasets/data_final.pkl'
    
#read in pkl file containing 
data = pd.read_pickle(path_FIFI)
data = pd.DataFrame(data = data)
data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', data.shape, '\n')

#clean data, combine 'sign/signal' and 'streetlight' 
data = data[data['FIFI_category'] != 'Community Walk']
data['FIFI_category'] = data['FIFI_category'].replace({'Needles/Dumping': 'Needles_Dumping', 'Sign/Signal': 'Sign_Signal_Light', 'Streetlight': 'Sign_Signal_Light'})

#we are only interested in FIFI requests that have a photo included with the request
print(data.shape)
data_photo = data[~data['Photo'].isna()]
print(data_photo.shape)

print(data_photo['FIFI_category'].value_counts())

Original data shape:
 (248815, 67) 

(248661, 67)
(183278, 67)
Needles_Dumping      40265
Other                34428
Grafitti             31749
Abnd_Vehicle         26656
Parking              25140
Pothole              13154
Sign_Signal_Light    11398
Clogged_Drain        427  
Vegetation           44   
Dead Animal          17   
Name: FIFI_category, dtype: int64


In [3]:
#create a directory for each category
save_path = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/photos/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Sign_Signal_Light']

for cat in cats_to_dl:
    if os.path.exists(os.path.join(save_path, cat)) == True:
        print(cat, " already exists")
    else:
        os.mkdir(os.path.join(save_path, cat))
        print(cat, " directory created")

Abnd_Vehicle  already exists
Grafitti  already exists
Needles_Dumping  already exists
Parking  already exists
Pothole  already exists
Sign_Signal_Light  already exists


In [4]:
#create function to get photo from url and save
def fetch_and_save(image_url_tupple):
    image_name, image_url = image_url_tupple
    #create file name for photo
    image_name = os.path.join(save_path_cat, image_name)  
    #get photo from url and save
    r = requests.get(image_url, allow_redirects=True)
    open(image_name, 'wb').write(r.content)

#do each category individually
for cat in cats_to_dl:
    print(cat)
    
    if len(os.listdir(os.path.join(save_path, cat))) > 0:
        print(cat, " already downloaded")
    else:
        cat_data = data_photo[data_photo['FIFI_category'] == cat].reset_index() 
        save_path_cat = os.path.join(save_path, cat)
        
        i = 0
        while i < cat_data.shape[0]:
            if i%50 == 0:
                print(i)
            #run on multiple threads for faster performance, save each photo with 'service_request_number' for later indexing
            urls = list(zip([cat+'_'+str(x)+'.jpeg' for x in cat_data['Service_Request_Number'].iloc[i:i+3]], cat_data['Photo'][i:i+3]))
            ThreadPool(3).imap_unordered(fetch_and_save, urls)
            i += 3

Abnd_Vehicle
Abnd_Vehicle  already downloaded
Grafitti
Grafitti  already downloaded
Needles_Dumping
Needles_Dumping  already downloaded
Parking
Parking  already downloaded
Pothole
Pothole  already downloaded
Sign_Signal_Light
Sign_Signal_Light  already downloaded


In [5]:
#check number of photos downloaded for each FIFI category compared to those listed in dataframe
for cat in cats_to_dl:
    print(cat)
    print('count in dataframe: ', data_photo[data_photo['FIFI_category'] == cat].shape[0])
    print('count in directory: ', len(os.listdir(os.path.join(save_path, cat))))
    print('\n')

Abnd_Vehicle
count in dataframe:  26656
count in directory:  26646


Grafitti
count in dataframe:  31749
count in directory:  31736


Needles_Dumping
count in dataframe:  40265
count in directory:  33146


Parking
count in dataframe:  25140
count in directory:  19162


Pothole
count in dataframe:  13154
count in directory:  7273


Sign_Signal_Light
count in dataframe:  11398
count in directory:  11398




In [6]:
#create directories for training and testing 
try:
    os.mkdir(os.path.join(save_path, 'cnn_cat_other'))
except OSError:
    print("directory already created")
try:
    os.mkdir(os.path.join(save_path, 'cnn_cat_other/training'))
except OSError:
    print("directory already created")
try:
    os.mkdir(os.path.join(save_path, 'cnn_cat_other/testing'))
except OSError:
    print("directory already created")


directory already created


In [7]:
#create directories for train/test for each FIFI category 
photo_categories = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Sign_Signal_Light']

train_dir = os.path.join(save_path, 'cnn_cat_other/training')
test_dir = os.path.join(save_path, 'cnn_cat_other/testing')

for cat in photo_categories:
    try:
        os.mkdir(os.path.join(train_dir, cat))
    except OSError:
        print('cannot make directory for train: ', cat)
    try:
        os.mkdir(os.path.join(test_dir, cat))
    except OSError:
        print('cannot make directory for test: ', cat)

In [8]:
#function to check file size (throw out empty files) and split data into training and testing based on a specified total number of data entries and split size
def split_data(source_dir, training_dir, testing_dir, size, split_size, seed):
    
    #check for non zero file size
    print('creating file list')
    files = [filename if os.path.getsize(os.path.join(source_dir, filename)) > 0 else print(filename + " is zero length, so ignoring.") for filename in os.listdir(source_dir)]
  
    #create entries of specified size and seed
    random.seed(seed)
    files_final = random.sample(files, size)
    #split into train and test based on specified split size
    training_length = int(len(files_final) * split_size)
    testing_length = int(len(files_final) - training_length)
    shuffled_set = random.sample(files_final, len(files_final))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    print('copying into train')
    for filename in training_set:
        this_file = os.path.join(source_dir, filename)
        destination = os.path.join(training_dir, filename)
        copyfile(this_file, destination)

    print('copying into test')
    for filename in testing_set:
        this_file = os.path.join(source_dir, filename)
        destination = os.path.join(testing_dir, filename)
        copyfile(this_file, destination)

In [9]:
#separate each FIFI category into train and test using split data function
for cat in cats_to_dl:
    print(cat)
    source_dir = os.path.join(save_path, cat)
    train_dir_cat = os.path.join(train_dir, cat)
    test_dir_cat = os.path.join(test_dir, cat)

    size = 7000
    split_size = .9
    seed = 39
    
    start_time = time.time()
    split_data(source_dir, train_dir_cat, test_dir_cat, size, split_size, seed)
    print("--- %s seconds ---" % (time.time() - start_time))


Abnd_Vehicle
creating file list
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

...

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into

copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into train
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copyi

copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying into test
copying in

In [14]:
#train CNN using images from each category besides 'other'

TRAINING_DIR = train_dir
training_datagen = ImageDataGenerator(rescale = 1./255)

VALIDATION_DIR = test_dir
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    class_mode='categorical')

model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit_generator(train_generator, epochs=15, validation_data = validation_generator, verbose = 1)

#model.save("rps.h5")


Found 37799 images belonging to 6 classes.
Found 4200 images belonging to 6 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________

KeyboardInterrupt: 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()